In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from dataloader import get_dataloaders_subword
import nltk
from transformer import Transformer,TransformerEncoder,TransformerDecoder
import utils
from utils_subword import load_sentencepiece_model,sp_detokenize_with_specials

import pickle
from tabulate import tabulate
import utils_subword
nltk.download('punkt')  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/wicaksonolxn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
BATCH_SIZE = 64# butuh lebih banyak update 
DATA_PATH = "dataset/"  
DATA_PATH_SPM = "dataset/spm_files"  
SAVE_DIR = "saved"
SRC_VOCAB_SIZE = 5000
TGT_VOCAB_SIZE = 5000
N_LAYERS = 2
N_HEADS = 1
D_MODEL = 128
FFN_HIDDEN = D_MODEL*4
DROPOUT = 0.1
EPOCHS = 300

sp = load_sentencepiece_model(DATA_PATH_SPM, model_prefix="spm_model")
train_loader, val_loader, test_loader = get_dataloaders_subword(
    sp=sp,
    data_path=DATA_PATH, 
    source_lang="min", 
    target_lang="eng", 
    batch_size=BATCH_SIZE, 
    device=device
)

encoder = TransformerEncoder(SRC_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
decoder = TransformerDecoder(TGT_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
best_model = Transformer(encoder,decoder,device,utils.PAD_TOKEN).to(device)
best_model.load_state_dict(torch.load(os.path.join(SAVE_DIR, "best_sp.pt")))
criterion = nn.CrossEntropyLoss(ignore_index=utils.PAD_TOKEN) 
print("Model initialized on:", device)
print("Loaded best model for testing!")


SentencePiece model loaded from dataset/spm_files/spm_model.model
TrainData - Max 'min' sentence length: 76
TrainData - Max 'eng' sentence length: 107
TestData - Max 'min' sentence length: 61
TestData - Max 'eng' sentence length: 75
ValidData - Max 'min' sentence length: 71
ValidData - Max 'eng' sentence length: 80
Number of examples in train_dataset: 800
Number of examples in valid_dataset: 100
Number of examples in test_dataset: 100
Model initialized on: cuda
Loaded best model for testing!


In [3]:
import os
import torch
import pickle
import utils
def evaluate_test(model, test_dataset, sp, device, criterion):
    model.eval()
    total_loss = 0.0
    all_bleu   = []
    with torch.no_grad():
        for i in range(len(test_dataset)):
            sample = test_dataset[i]
            src_token_ids = sample["src"]
            tgt_token_ids = sample["tgt"]
            # If tokens are tensors, convert them to lists.
            if torch.is_tensor(src_token_ids):
                src_token_ids = src_token_ids.tolist()
            if torch.is_tensor(tgt_token_ids):
                tgt_token_ids = tgt_token_ids.tolist()
                
            src_tensor = torch.LongTensor(src_token_ids).unsqueeze(0).to(device)
            tgt_tensor = torch.LongTensor(tgt_token_ids).unsqueeze(0).to(device)
            
            # Forward pass through the model (teacher forcing for target input).
            output, _ = model(src_tensor, tgt_tensor[:, :-1])  # shape: [1, seq_len-1, vocab_size]
            vocab_size = output.shape[-1]
            output_2d = output.view(-1, vocab_size)              # [seq_len-1, vocab_size]
            tgt_2d    = tgt_tensor[:, 1:].contiguous().view(-1)    # [seq_len-1]
            loss = criterion(output_2d, tgt_2d)
            total_loss += loss.item()
            
            # Use SentencePiece detokenization to get human-readable sentences.
            ref_text = sp_detokenize_with_specials(sp, tgt_token_ids)
            pred_ids = output[0].argmax(dim=1).tolist()            # [seq_len-1]
            hyp_text = sp_detokenize_with_specials(sp, pred_ids)
            
            bleu_score = utils.get_bleu(hyp_text.split(), ref_text.split())
            all_bleu.append(bleu_score)
            
    avg_loss = total_loss / len(test_dataset)
    avg_bleu = sum(all_bleu) / len(all_bleu)
    return avg_loss, avg_bleu

# Example usage:
test_loss, test_bleu = evaluate_test(best_model, test_loader, sp, device, criterion)
print(f"Test Loss = {test_loss:.4f} | BLEU = {test_bleu:.2f}")


Test Loss = 4.7227 | BLEU = 0.28


In [4]:
import os
import pickle
import torch
from translation import translate_sentence_piece  
num_samples_to_translate = 20
for i in range(num_samples_to_translate):
    sample = test_loader[i]  
    src_token_ids = sample["src"]
    tgt_token_ids = sample["tgt"]
    
    if torch.is_tensor(src_token_ids):
        src_token_ids = src_token_ids.tolist()
    if torch.is_tensor(tgt_token_ids):
        tgt_token_ids = tgt_token_ids.tolist()
    
    src_text = sp_detokenize_with_specials(sp, src_token_ids)
    real_target_text = sp_detokenize_with_specials(sp, tgt_token_ids)
    
    predicted_translation, predicted_tokens = translate_sentence_piece(
        token_ids=src_token_ids,
        sp=sp,
        model=best_model,
        device=device,
        max_len=utils.MAX_SENT_LEN,
    )
    
    print("_________________________________________________")
    print(f"|                 SRC NO.{i+1}                  |")
    print("_________________________________________________")
    print(f"Source: {src_text}")
    print(f"Predicted Token : {predicted_tokens}")
    print(f"Predicted Translation: {predicted_translation}")
    print(f"Real Target: {real_target_text}\n")
    print("_______________________________________________")


_________________________________________________
|                 SRC NO.1                  |
_________________________________________________
Source: kangkuangnyo lumayan tapi kapitiang saus padangnyo mangecewaan kami diagiah kapitiang yang kopong akhiar kami ndak makan kapitiang dan dibaliakan
Predicted Token : [1, 910, 4994, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987, 158, 137, 4993, 47, 4987]
Predicted Translation: they withere withere withere withere withere withere withere withere withere withere withere withe

In [5]:
for i in range(len(test_loader)):
    if i<1:
        sample = test_loader[i] 
        src_token_ids = sample["src"].tolist()
        tgt_token_ids = sample["tgt"].tolist()
        src_text = utils.detokenize(src_token_ids, input_lang_dic)
        tgt_text = utils.detokenize(tgt_token_ids, output_lang_dic)
        print(f"src: {src_text}\ntgt: {tgt_text}\n")

NameError: name 'input_lang_dic' is not defined

In [ ]:
print("Index to word mapping (first 10):")
for i in range(10):
    print(i, output_lang_dic.index2word[i])
print("Dictionary size:", len(input_lang_dic.word2index))

Index to word mapping (first 10):
0 PAD
1 SOS
2 EOS
3 UNK
4 enjoy
5 instalment
6 for
7 up
8 to
9 months
Dictionary size: 3880


In [ ]:
print("Special tokens in the dictionary:")
for idx in range(4):
    print(idx, input_lang_dic.index2word[idx])

Special tokens in the dictionary:
0 PAD
1 SOS
2 EOS
3 UNK
